# error message text:

C:\Users\zella\Desktop\Personal\Education\University of Kentucky\spatial_mapping\Spatial_Mapping_Training\Zella\Dartmouth_csv_test_catchment_data\src
please locate the chrome driver file in ./input_folder\chromedriver.exe (TEST 1)
you can download the driver file from https://sites.google.com/chromium.org/driver/ (TEST 1)
Traceback (most recent call last):
  File "C:\Users\zella\Desktop\Personal\Education\University of Kentucky\spatial_mapping\Spatial_Mapping_Training\Zella\z_annotated_cifpull.py", line 706, in <module>
    comp_data4AGOL()
  File "C:\Users\zella\Desktop\Personal\Education\University of Kentucky\spatial_mapping\Spatial_Mapping_Training\Zella\z_annotated_cifpull.py", line 585, in comp_data4AGOL
    env = gen_env_data()
  File "C:\Users\zella\Desktop\Personal\Education\University of Kentucky\spatial_mapping\Spatial_Mapping_Training\Zella\z_annotated_cifpull.py", line 437, in gen_env_data
    FCC = fcc(state=s, download_path = dl_path, chrome_driver_path = drive_file)
  File "C:\Users\zella\Desktop\Personal\Education\University of Kentucky\spatial_mapping\Spatial_Mapping_Training\Zella\z_annotated_ciftools.py", line 1435, in __init__
    self.gen_df()
  File "C:\Users\zella\Desktop\Personal\Education\University of Kentucky\spatial_mapping\Spatial_Mapping_Training\Zella\z_annotated_ciftools.py", line 1492, in gen_df
    file_path = self.file_path # ZELLA NOTE: AttributeError: 'fcc' object has no attribute 'file_path'
AttributeError: 'fcc' object has no attribute 'file_path'


In [15]:
# import everything
import pandas as pd
import requests
import re
import numpy as np
from bs4 import BeautifulSoup
import time
import os
from io import BytesIO
from zipfile import ZipFile
import urllib.request
import glob
import sys # can't tell if this is in the Tools file
from selenium import webdriver
from selenium.webdriver.common.by import By

In [67]:
# fcc class

class fcc:
    def __init__(self, state = 'KY', chrome_driver_path = None, download_path = None,  remove_download = False, file_path = None):
        response = requests.get('https://www.fcc.gov/general/broadband-deployment-data-fcc-form-477')
        soup = BeautifulSoup(response.content, "html.parser")
        self.data_url = soup.find_all(href=re.compile(state))[-1]['href']
        self.download_data(state2 = state, chrome_driver_path = chrome_driver_path, download_path = download_path)
        self.gen_df()
        self.file_path = r'C:\Users\zella\Desktop\Personal\Education\University of Kentucky\spatial_mapping\Spatial_Mapping_Training\Zella\Dartmouth_csv_test_catchment_data' # zella note: temporary, for testing
        if remove_download: 
            os.remove(self.file_path)



    def download_data(self, state2 = 'KY', chrome_driver_path = None, download_path = None):
        # from selenium.webdriver.support.ui import WebDriverWait as wait
        # from selenium.webdriver.common.keys import Keys
        # from selenium.webdriver.support.ui import Select
        from selenium import webdriver
        from selenium.webdriver.common.by import By

        if chrome_driver_path == None:
            chrome_driver_path = 'chromedriver'
            if sys.platform.lower()[:3] == 'win':
            	chrome_driver_path += '.exe'
            chrome_driver_path = os.path.join(os.getcwd() , 'input_folder', chrome_driver_path)
            print(chrome_driver_path)
        try:
            driver = webdriver.Chrome(chrome_driver_path) # Zella note: seems like this line may be causing errors
            driver.close()
        except:
            print(f'please locate the chrome driver file in {chrome_driver_path} (TEST 1)') # Zella note: adding test text to distinguish NOTE: this is being triggered
            print('you can download the driver file from https://sites.google.com/chromium.org/driver/ (TEST 1)')  # Zella note: adding test text to distinguish
        else:
            if download_path:
                chrome_options = webdriver.ChromeOptions()
                prefs = {'download.default_directory' : download_path}
                chrome_options.add_experimental_option('prefs', prefs)
                driver = webdriver.Chrome(chrome_driver_path, chrome_options = chrome_options)
            else:
                driver = webdriver.Chrome(chrome_driver_path)
            driver.get(self.data_url)
            time.sleep(5)
            driver.find_elements(By.CLASS_NAME, 'btn')[1].click()

            t = 0

            while t == False:
                time.sleep(5)
                t = os.path.isfile(f'{download_path}\\{state2}-Fixed-Jun2021.zip')
                t = len(glob.glob(os.path.join(download_path, '*.zip'), recursive = True)) > 0
                print(t)
                print(f'Waiting on {state2} broadband data...')
            else:
                print(f'{state2} broadband data ready')

            driver.close()

            if download_path == None:
                print('check your download folder for the file')
            else:
                file_path = glob.glob(os.path.join(download_path, '*.zip'), recursive = True)[-1]
                self.file_path = file_path
                print(file_path)

    def gen_df(self):
        file_path = r'C:\Users\zella\Desktop\Personal\Education\University of Kentucky\spatial_mapping\Spatial_Mapping_Training\Zella\Dartmouth_csv_test_catchment_data.zip' # ZELLA NOTE: AttributeError: 'fcc' object has no attribute 'file_path'
        with ZipFile(file_path) as my_zip_file:
            df = pd.read_csv(my_zip_file.open(my_zip_file.namelist()[0]))
            df = df[['StateAbbr','BlockCode','MaxAdDown','MaxAdUp']]
            print(file_path)
            self.fcc_data = df
            my_zip_file.close()

In [68]:
f = fcc()

C:\Users\zella\Desktop\Personal\Education\University of Kentucky\spatial_mapping\Spatial_Mapping_Training\Zella\input_folder\chromedriver.exe


C:\Users\zella\AppData\Local\Temp\ipykernel_6316\865820118.py:30: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_path) # Zella note: seems like this line may be causing errors
C:\Users\zella\AppData\Local\Temp\ipykernel_6316\865820118.py:42: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_path)


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [54]:
# establishing chromedriver path
chrome_driver_path = 'chromedriver'
if sys.platform.lower()[:3] == 'win':
    chrome_driver_path += '.exe'
chrome_driver_path = os.path.join(os.getcwd() , 'input_folder', chrome_driver_path)
# C:\Users\zella\Desktop\Personal\Education\University of Kentucky\spatial_mapping\Spatial_Mapping_Training\Zella\input_folder

In [58]:
fcc.gen_df()

TypeError: gen_df() missing 1 required positional argument: 'self'